# SVM: LinearSVC

#### Eerste getrainde model

Dependencies installeren (Feather & Multithreading)

In [1]:
!pip install pyarrow
!pip install --upgrade pathos

     |████████████████████████████████| 17.8 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 219 kB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.5 MB/s eta 0:00:010:01
     |████████████████████████████████| 2.4 MB 10.3 MB/s eta 0:00:01
  Created wheel for pathos: filename=pathos-0.2.6-py3-none-any.whl size=77675 sha256=1841f21afd40d11645647df985559db169273877e8cc96352f4ddd558567f884
  Stored in directory: /home/aivnevel/.cache/pip/wheels/83/12/43/a7b335eac30d213e8cbe768bff7456ef4bc76351769864dd79
  Created wheel for multiprocess: filename=multiprocess-0.70.10-py3-none-any.whl size=125838 sha256=ff57943bc0e485db2311eb7ea2d5db328e8cc523060717e5c982a735aeaf475e
  Stored in directory: /home/aivnevel/.cache/pip/wheels/f4/c7/82/0bcdc3506e6a8365963db24f477b81693438d4b9165c0757b7
Successfully built pathos multiprocess


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd
import numpy as np
import os

DIR = './Datasets/local/'

names = []
dfs = {}
for filename in os.listdir(DIR):
    dfs[filename] = pd.read_feather(DIR + filename)
    names.append(filename)

# Models trained with balanced datasets

In [9]:
import pathos.pools as pp
import multiprocessing as mp
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

def create_SVC_model(df):
    df_result = df['label']
    df = df.drop(['label'], axis=1)
    clf = make_pipeline(MinMaxScaler(), LinearSVC(dual=False, tol=1e-5, max_iter=64000, class_weight='balanced'))
    clf.fit(df, df_result)
    return clf

In [ ]:
p = pp.ProcessPool(mp.cpu_count())
models_balanced = p.map(lambda key: create_SVC_model(dfs[key]), dfs.keys())
p.close()
p.join()
p.clear()

In [10]:
model = create_SVC_model(dfs['cicdos2017'])

In [12]:
df_test = dfs[names[0]]
df_testresult = df_test['label']
df_test = df_test.drop(['label'], axis=1)
y_test_clf_model = model.predict(df_test)
print("\t\t", "Weighted average:", accuracy_score(df_testresult, y_test_clf_model))
print("\t\t", "F1 score:", f1_score(df_testresult, y_test_clf_model, average='binary'))

		 Weighted average: 0.6279268622264754
		 F1 score: 0.6364474612664015


In [ ]:
from joblib import dump

it = 0
for model in models_balanced:
    dump(model, './models/svm/balanced/fully_trained/'+ names[it] + '.joblib') 
    it+=1

In [ ]:
from joblib import load

clf = {}
it = 0
for name in names:
    clf[name] = load('./models/svm/balanced/fully_trained/'+ names[it] + '.joblib') 
    it+=1

# Accuracy & F1 Score
Accuracy:
    Accuracy = TP / (TP + FP + FN + TN) 
F1 score:
    F1 = TP/ (TP + 1/2 * (FP + FN))

In [ ]:
from sklearn.metrics import f1_score

for key, model in clf.items():
    print(key,":")
    for j, path2 in dfs.items():
        df_test = dfs[j]
        df_testresult = df_test['label']
        df_test = df_test.drop(['label'], axis=1)
        y_test_clf_model = model.predict(df_test)
        print("\t", j,":")
        print("\t\t", "Weighted average:", accuracy_score(df_testresult, y_test_clf_model))
        print("\t\t", "F1 score:", f1_score(df_testresult, y_test_clf_model, average='binary'))
    print("----------------------------------------------------------------------------------------------")